<h1 align ="center"><img src="https://media.giphy.com/media/YjPhjzR0z8qpqtPFyx/giphy.gif" style = "height:38px"/> Procesamiento de los Datasets con Pandas <img src="https://media.giphy.com/media/A8ZKGHwM5lWpSqd8rX/giphy.gif" style = "height:38px"/></h1 >


Las tarjetas de FiguRace se generan a partir de un conjunto de datos que deberán ser preparados previamente. Como parte del proyecto se deberá desarrollar un cuaderno de Jupyter Notebook que extraiga los datos necesarios para el juego de alguno de los datasets provistos y muestre en forma detallada el proceso realizado con los mismos . En el Anexo II se detallan los datasets que deberán utilizar para la aplicación y las modificaciones necesarias para luego guardarlo en un nuevo archivo en formato CSV. El juego deberá cargar esos archivos CSV para funcionar


<table>
<tr>
<td> <h5 align="center"><img src="https://media.giphy.com/media/F3IsWfsR1JMyNmL44Z/giphy.gif" style = "height:38px" /> Spotify 2010 - 2019 Top 100 <img  src="https://media.giphy.com/media/F3IsWfsR1JMyNmL44Z/giphy.gif" style = "height:38px" /></h5> </td> <td> <h5 align="center"><img src="https://media.giphy.com/media/W55QKlBohHZS9RGOWJ/giphy.gif" style = "height:38px" /> FIFA-21 Complete <img  src="https://media.giphy.com/media/W55QKlBohHZS9RGOWJ/giphy.gif" style = "height:38px" /></h5> </td><td> <h5 align="center"><img src="https://media.giphy.com/media/YOk7USZ9k8yF4R0yn3/giphy.gif" style = "height:38px" /> Lagos Argentina - Hoja 1 <img  src="https://media.giphy.com/media/YOk7USZ9k8yF4R0yn3/giphy.gif" style = "height:38px" /></h5> </td>


</tr>
<tr>
<td>
 
Deberá adaptar los datos de la siguiente forma: <br>

- Poner en `title case` los géneros musicales excepto las siglas EDM, DFW, UK, R&B y LGBTQ+ que deben ir en mayúsculas. Por ejemplo `dfw rap` debe ser transformado a `DFW Rap`.<br>

- Considerar también la excepción `k-pop` que debe ser transformada a `K-Pop`.<br>

- Se utilizarán como datos de las tarjetas `Top Genre`, `Year Released`, `BPM`, `Top Year` y `Artist Type`. Como dato a adivinar se utilizará `Artist`. Descartar el resto de las columnas.<br>

- El archivo resultante deberá tener las siguientes columnas (en este orden específico): `Top Genre`, `Artist Type`, `Year Released`, `Top Year`, `BPM` y `Artist`<br>

</td>
<td>
 
Deberá adaptar los datos de la siguiente forma:<br>

- Reemplazar `Potential` por la siguiente escala conceptual:<br>
  - Regular: Menos de 60<br>
  - Bueno: Entre 60 y 79 (inclusive)<br>
  - Muy bueno: Entre 80 y 89 (inclusive)<br>
  - Sobresaliente: Desde 90 en adelante.<br>
- Reemplazar el valor de `Position` por las posiciones en español. Por ejemplo `LB|CB` debe ser reemplazado por `Defensor izquierdo|Defensor central`<br>
- Se utilizarán como datos de las tarjetas: `Age`, `Nationality`, `Position`, `Team` y `Potential`. Como dato a adivinar se utilizará `Name`. Descartar el resto de las columnas.<br>
- El archivo resultante deberá tener las siguientes columnas (en este orden específico): `Team`, `Nationality`, `Position`, `Age`, `Potential` y `Name`.<br>
 
</td>
 <td>
 

Deberá adaptar los datos de la siguiente forma:<br>

- Transformar las coordenadas en la columna `Coordenadas` a grados decimales.<br>
- Se utilizarán como datos de las tarjetas: `Ubicación`, `Superficie (km²)`, `Profundidad máxima (m)`, `Profundidad media (m)`, `Coordenadas`. Como dato a adivinar se utilizará `Nombre`. Descartar el resto de las columnas.<br>
- El archivo resultante deberá tener las siguientes columnas (en este orden específico): `Ubicación`, `Superficie (km²)`, `Profundidad máxima (m)`, `Profundidad media (m)`, `Coordenadas` y `Nombre`.<br>
 
</td>
</tr>
 
</table>

### Librerias a utilizar


In [ ]:
import os
import pandas as pd

### Constantes a utilizar 

- En `POTENTIAL_TABLE_FIFA` almacenamos los datos que vamos a remplazar de la columna `potencial`
- En `POSITION_TABLE_FIFA` almacenamos los datos que vamos a traducir de la columna `position` 
- En `UPPER_GENDERS_SPOTIFY` almacenamos los datos a sustituir de la columna `top genre`

In [122]:
POTENTIAL_TABLE_FIFA = {
    90: 'Sobresaliente',
    80: 'Muy bueno',
    60: 'Bueno',
    -1: 'Regular'
}

POSITION_TABLE_FIFA = {
    'ST': 'Delantero',
    'CM': 'Volante',
    'CDM': 'Medio centro defensivo',
    'LB': 'Lateral izquierdo',
    'GK': 'Portero',
    'LM': 'Volante izquierdo',
    'RM': 'Volante derecho',
    'CAM': 'Volante ofensivo',
    'LW': 'Extremo izquierdo',
    'LWB': 'Lateral izquierdo ofensivo',
    'CB': 'Defensor central',
    'RB': 'Lateral derecho',
    'RW': 'Extremo derecho',
    'RWB': 'Lateral ofensivo derecho',
    'CF': 'Media punta'
}

UPPER_GENDERS_SPOTIFY = ["EDM", "DFW", "UK", "R&B", "LGBTQ+"]

## Funciones que transforman columnas

### Potencial Fifa

Recibimos el potencial del archivo (como estamos trabajando con archivos, nos pasa un string), y lo vamos comparando con un diccionario (que en las claves tiene su potencial en numeros y en valor, su clasificación como jugador) `POTENTIAL_TABLE_FIFA`, cuando el potencial pasado por parametro es mayor, lo remplazo por el valor del diccionario

In [123]:
def potential_replace(potential):
    compare_potential = int(potential)
    for potential_player in POTENTIAL_TABLE_FIFA:
        if compare_potential >= potential_player:
            potential = POTENTIAL_TABLE_FIFA[potential_player]
            break
    return potential

### Remplazar posición Fifa

- Recibimos la posicion por parametro y la pasamos a español (remplazado el dato con el de una tabla que armamos mas arriba )

- En `positions` generamos una lista dividida por `|` para poder modificar cada posición por separado.
- Por ultimo, remplazamos los datos con el for y las volvemos a concatenar con `|`

In [124]:
def position_replace(position):
    positions = position.split('|')
    position = '|'.join([POSITION_TABLE_FIFA[acronym] for acronym in positions])
    return position

### Mayusculas-Minusculas para Spotify

- `genders` Con la frase que tenemos (`sentence`), generamos una lista de palabras para poder trabajar en cada una de ellas por separado

- Recorremos la lista
  - Si se encuentra en la lista, la convertimos toda a mayuscula
  - Sino usamos `title()` para pasar la primera a mayuscula

In [125]:
def upper_words(sentence):
    genders = sentence.split()
    for index,gender in enumerate(genders):
        genders[index] = ( gender.upper() if gender.upper() in UPPER_GENDERS_SPOTIFY else gender.title())
    sentence = " ".join(genders)
    return sentence

### Convertir coordenadas grados a decimales

Si la tenemos en el sistema tradicional de grados, minutos y segundos es necesario convertirla previamente.

Para llevar manualmente de grados a decimal sería:

- `(23° 08' 06'' N)` = `(23 + (08 / 60) + (06 / 3600)) * 1` = `23.134999`
- `(82° 21' 34'' W)` = `(82 + (21 / 60) + (34 / 3600)) * -1` = `-82.359444`

- `sign` Guardamos el signo dependiedo de si recibimos la latitud o longitud como parametro
- `"50°14'53""S 72°38'43""O"` dato recibido por parametro
- En la linea 3 separamos los grados de las cordenadas
- En la linea 4 separamos los minutos y los segundos de la cordenada.
- Por ultimo aplicamos la ecuación y retornamos el resultado.
- `round` nos permite limitar el uso de decimales


In [126]:
def rebase_coord(coord, n_decimals = 5):
    sign = -1 if 'S' in coord or 'O' in coord else 1
    degree, coord = coord[:-2].split('°')
    min, sec = coord.split('\'')
    dd = sign * (int(degree) + int(min)/60 + int(sec)/3600)
    return str(round(dd, n_decimals)) + '°'

Como tenemos la misma operación para dos partes de nuestras cordenadas, las separamos, las trabajamos por separado, las concatenamos y por ultimo las retornamos

In [127]:
def transform_coords(coords) :
    latitude, longitude = coords.split()
    coords = rebase_coord(latitude) + ' ' + rebase_coord(longitude)
    return coords

## Configuraciones a aplicar 

Tenemos un diccionario en el que sus claves son los nombres de los datasets a modificar y sus valores son las caracteristicas que le queremos aplicar a cada dataset

- `order` Son las columnas que quiero consevar y el orden preestablecido por el usuario
- `translation` La traduccion del encabezado (opcional)
- `functions` Las funciones que vamos a aplicar (las clave es la columna y su valor es la funcion que queremos aplicar)
- `name` El nuevo nombre de los datasets

In [128]:

DATASETS = {                  
    'FIFA-21_Complete.csv':{
        'order': ["team", "nationality", "position", "age", "potential" ,"name"],
        'translation': ['Equipo', 'Nacionalidad', 'Posición', 'Edad', 'Potencial', 'Nombre'],
        'functions': {
            "potential": potential_replace,
            "position": position_replace
            } ,
        'name':"fifa.csv"     
    },
    'Lagos_Argentina - Hoja_1.csv':{
        'order': ["Ubicación", "Superficie (km²)", "Profundidad máxima (m)", "Profundidad media (m)", "Coordenadas"],
        'functions': {
            "Coordenadas": transform_coords
            },  
        "name":'lakes.csv'  
    },
    'Spotify_2010-2019_Top_100.csv':{
        'order': ["top genre", "artist type", "year released", "top year", "bpm" ,"artist"],
        'translation': ['Top genero', 'Tipo artista', 'Año lanzamiento','Mejor año', 'BPM', 'Artista'],
        'functions': {
            "top genre":upper_words
            },
        'name':'spotify.csv'       
    }
}


### Rutas a utilizar

- `PATH_BASE` Ruta base
- `PATH_SOURCE` Ruta de donde leemos los datasets
- `PATH_PROSSED` Ruta en donde vamos a procesar los datasets

En caso de que no exista la carpeta, la generamos con `makedirs`

In [129]:
PATH_BASE = os.path.dirname(os.path.dirname(__file__))
PATH_SOURCE = os.path.join(PATH_BASE, "base_datasets")
PATH_PROSSED = os.path.join(PATH_BASE, "processed_datasets")
if not os.path.exists(PATH_PROSSED):
    os.makedirs(PATH_PROSSED, exist_ok=True)

### Procesamiento de un Dataset

- Recibimos `file_name` que contiene el nombre del archivo, preguntamos si el nombre se encuentra en nuestro diccionario (asi decidimos si lo procesamos o no)
- `file_path` Nos quedamos la ruta en donde se encuentra nuestro df
- En `config` como su palabra lo indica, nos quedamos con las configuraciones del df (data frame) a leer  
- `processed_path` contiene la dirección con el nuevo nombre de nuestro archivo
- Abrimos el archivo en modo escritura
    - Dentro del read_csv tenemos:
        - El archivo, el delimiter en "sep" que python detecta el separador.
        - Las columnas a guardar, el tipo de datos que vamos a leer
        - E ignoramos las filas que puedan tener algun tipo de error, y utilizamos `error_bad_lines=False` 
    - Y tomamos una excepcion en caso de que la ruta no exista
- En df.dropna eliminamos las columnas vacias
- Aplicamos la config `order`
- Recorremos la config `functions` y aplicamos su funcion a la columna marcada por su respectiva clave
- Preguntamos si tenemos la config `translation`.
    - en df.rename() lo que estamos haciendo es mandarle una clave y un valor para que lo remplzace por dicha clave (la traducción)
    - En el parametro axis posee 1 ya que solo trabajamos con el encabezado
- Reemplazamos todas las celdas vacias por `Desconocido`

In [130]:
def process_dataset(file_name):
    if file_name not in DATASETS:
        return
    file_path = os.path.join(PATH_SOURCE, file_name)
    config = DATASETS[file_name]
    processed_path = os.path.join(PATH_PROSSED, config['name'])
    try:
        with open(file_path, mode='r', encoding="UTF-8") as file:
            df = pd. read_csv(file, sep=None, engine="python",
                              usecols=(config['order']), dtype=str,error_bad_lines=False)
    except FileNotFoundError:
        print('No existe la ruta', PATH_SOURCE)
        return
    df.dropna(how="all", inplace=True)

    df = df[config['order']]

    for columna, function in config['functions'].items():
        df[columna] = df[columna].apply(function)
    df.fillna('Desconocido',  inplace=True)

    df.to_csv(processed_path, mode='w', index=False)

### Busqueda y Procesamiento

Tratamos de leer la direcciones y en caso de tener una excepción (que pude ocurrir que mi ruta no coincida con mi carpeta que es `FileNotFoundError` o que mi ruta no sea un directorio `NotADirectoryError`) imprimimos un mensaje con su correspondiente error.

- `listdir` nos genera una lista con los nombres de todos los archivos 
- Iteramos sobre esta lista ya aplicamos nuestra funcion a cada elemento

In [131]:
try:
    names_files = os.listdir(PATH_SOURCE)
    for file_name in names_files:
        process_dataset(file_name)
except FileNotFoundError:
    print('No existe la ruta', PATH_SOURCE)
except NotADirectoryError:
    print('La ruta no es un directorio ', PATH_SOURCE)